# PCG 自洽场 (SCF) 求解流程概述
- 统一通过 `%run` 载入 `basis_func_with_kinetic.ipynb`、`Hartree.ipynb`、`Vps_lc.ipynb`、`Vps_nl.ipynb`、`Vex.ipynb` 中已经实现的平面波基组、Hartree/赝势/交换能模块。
- 在 K 空间使用平面波基展开波函数，并通过 K 空间势矩阵（局域与非局域赝势）以及由实时密度计算得到的 Hartree 与交换-相关势构造 KS Hamiltonian。
- 以 `generate_G_vectors` 返回的平面波为初始波函数，采用预条件共轭梯度 (PCG) 迭代求解 KS 方程最低本征态，同时迭代密度以实现 SCF。

In [359]:
%run basis_func_with_kinetic.ipynb
%run Hartree.ipynb
%run Vps_lc.ipynb
%run Vps_nl.ipynb
%run Vex.ipynb
%run collect_ens.ipynb

import numpy as np
from dataclasses import dataclass
from typing import Dict, Tuple

np.set_printoptions(precision=6, suppress=True)

Plane waves: 113
FFT grid Nk: 32
First ten eigenvalues (Hartree):
[-25.318211 -21.242306 -21.242306 -21.242099 -17.665538 -17.665538
 -17.665221 -16.131219 -15.209956 -15.087233]


In [360]:
TOL = 1e-8

@dataclass
class PlaneWaveBasis:
    """以平面波展开为核心的基组描述，包含晶格、G 向量、FFT 网格等全部信息。"""
    a_vecs: np.ndarray
    b_vecs: np.ndarray
    Omega: float
    Gs: np.ndarray
    energies: np.ndarray
    coeffs_int: list
    kvec: np.ndarray
    Nk: int
    r_grid: np.ndarray
    r_points: np.ndarray
    dV: float
    pw_matrix: np.ndarray

def build_plane_wave_basis(a0=4.05, Ecut_eV=80.0, Nk=16, kvec=None):
    """
    输入晶格常数、动能截断、FFT 网格数以及可选 k 向量，返回 PlaneWaveBasis。
    输出结构里同时提供实/倒空间基矢、所有 G 向量、对应能量和实空间采样矩阵，供后续波函数转换与积分使用。
    """
    a_vecs = primitive_fcc(a0)
    b_vecs, Omega = reciprocal_vectors(a_vecs)
    Gs, coeffs, energies = generate_G_vectors(b_vecs, Ecut_eV, kvec=kvec)
    if len(Gs) == 0:
        raise RuntimeError('未找到满足截断的 G 向量，请提高 Ecut_eV。')
    if kvec is None:
        kvec = np.zeros(3)

    us = np.arange(Nk) / Nk
    U, V, W = np.meshgrid(us, us, us, indexing='ij')
    r = (U[...,None] * a_vecs[0] + V[...,None] * a_vecs[1] + W[...,None] * a_vecs[2])
    r_points = r.reshape(-1, 3)
    dV = Omega / (Nk**3)

    k_plus_G = Gs + kvec
    phase = r_points @ k_plus_G.T
    pw_matrix = np.exp(1j * phase) / np.sqrt(Omega)

    return PlaneWaveBasis(
        a_vecs=a_vecs,
        b_vecs=b_vecs,
        Omega=Omega,
        Gs=Gs,
        energies=energies,
        coeffs_int=coeffs,
        kvec=np.array(kvec, dtype=float),
        Nk=Nk,
        r_grid=r,
        r_points=r_points,
        dV=dV,
        pw_matrix=pw_matrix,
    )

def coeffs_to_real(psi_coeffs: np.ndarray, basis: PlaneWaveBasis) -> np.ndarray:
    """
    输入平面波系数与对应基组，将系数乘以 pw_matrix 得到实空间波函数，并返回 Nk^3 网格的三维数组。
    """
    psi_flat = basis.pw_matrix @ psi_coeffs
    return psi_flat.reshape((basis.Nk, basis.Nk, basis.Nk))

def real_to_coeffs(psi_r: np.ndarray, basis: PlaneWaveBasis) -> np.ndarray:
    """
    输入实空间波函数网格与基组信息，先展平再乘以共轭转置的 pw_matrix，并乘体积元得到平面波系数向量。
    """
    psi_flat = psi_r.reshape(-1)
    return basis.pw_matrix.conj().T @ (psi_flat * basis.dV)

def normalize_state(psi_coeffs: np.ndarray) -> np.ndarray:
    """
    将输入的波函数系数按 L2 范数归一化，若范数过小则报错，输出为单位范数向量。
    """
    norm = np.linalg.norm(psi_coeffs)
    if norm < 1e-14:
        raise RuntimeError('波函数范数过小。')
    return psi_coeffs / norm

def project_out_states(vec, states, mode, id) -> np.ndarray:
    """
    输入待投影向量与一组任意态，逐一减去在这些态上的投影，返回与这些态正交的向量。
    """
    result = vec.copy()
    if mode == 'one':
        result -= np.vdot(states, result) * states
        return result
    elif mode == 'multi':
        for idy in range(len(states)):
            state = states[:, idy]
            result -= np.vdot(state, result) * state
        return result
    elif mode == 'notself':
        for idy in range(len(states)):
            state = states[:, idy]
            if np.array_equal(state, vec):
                continue
            result -= np.vdot(state, result) * state
        return result



In [361]:
def compute_density(psi_coeffs: np.ndarray, basis: PlaneWaveBasis, nelec: float = 1.0) -> np.ndarray:
    """
    将平面波系数转换为实空间波函数并计算 |ψ|^2，随后按电子数 `nelec` 归一化，输出实空间密度网格。
    """
    psi_r = coeffs_to_real(psi_coeffs, basis)
    rho = np.abs(psi_r)**2 * basis.dV
    integral = np.sum(rho) 
    if integral < 1e-8:
        raise RuntimeError('密度积分为 0。')
    rho *= nelec / integral
    return rho

def fft_density(rho: np.ndarray) -> np.ndarray:
    """
    输入实空间密度张量，执行正规化 FFT 得到 G 空间密度系数，输出大小等同于输入网格。
    """
    rho_G = np.fft.fftn(rho) / rho.size
    return rho_G

def build_G_grid(basis: PlaneWaveBasis) -> np.ndarray:
    """
    根据基组 FFT 网格构造每个实空间点对应的倒空间向量 G(r)，输出形状 (Nk,Nk,Nk,3) 的张量。
    """
    Nk = basis.Nk
    freq = np.fft.fftfreq(Nk) * Nk
    mx, my, mz = np.meshgrid(freq, freq, freq, indexing='ij')
    Ggrid = (mx[...,None] * basis.b_vecs[0] + my[...,None] * basis.b_vecs[1] + mz[...,None] * basis.b_vecs[2]) / Nk
    return Ggrid


def lda_exchange_correlation(rho: np.ndarray):
    """
    包装 `lda_exchange_potential`，输入实空间密度返回局域交换能密度和势，用于能量分解和势能构造。
    """
    eps_x, v_x = lda_exchange_potential(rho)
    return eps_x, v_x

In [362]:
def vector_key(vec: np.ndarray) -> Tuple[int, int, int]:
    """
    将连续 G 向量按 TOL 网格四舍五入成整数三元组，便于字典查找局域赝势矩阵元。
    """
    scaled = np.round(vec / TOL).astype(int)
    return tuple(scaled.tolist())

def apply_local_potential(psi_coeffs: np.ndarray, basis: PlaneWaveBasis, vdict: Dict[Tuple[int,int,int], float]) -> np.ndarray:
    """
    对输入系数向量逐个累积 V_loc(G-G')*ψ_G'，输出局域赝势作用后的新系数向量。
    """
    result = np.zeros_like(psi_coeffs, dtype=complex)
    for i, Gi in enumerate(basis.Gs):
        acc = 0.0j
        for j, Gj in enumerate(basis.Gs):
            diff_key = vector_key(Gi - Gj)
            Vij = vdict.get(diff_key)
            if Vij is None:
                continue
            acc += Vij * psi_coeffs[j]
        result[i] = acc
    return result



In [363]:
def apply_real_space_potential(psi_coeffs: np.ndarray, basis: PlaneWaveBasis, v_r: np.ndarray) -> np.ndarray:
    """
    输入平面波系数与实空间势能网格，先转到实空间做逐点乘法，再映射回系数表示，输出 V(r)ψ 的平面波系数。
    """
    psi_r = coeffs_to_real(psi_coeffs, basis)
    vpsi_r = v_r * psi_r
    return real_to_coeffs(vpsi_r, basis)

def build_preconditioner(basis: PlaneWaveBasis, energy_shift: float = 1.0) -> np.ndarray:
    """
    依据动能本征值构造 1/(ε_G+shift) 型预条件子数组，低能平面波以较大权重加速 PCG 收敛。
    """
    diag = basis.energies + energy_shift
    diag = np.where(diag < 1e-3, 1e-3, diag)
    return 1.0 / diag

def update_states(psi, D_prime, H_matrix, theta0 = np.pi/300):
    F_0 = psi @ H_matrix @ psi.T
    psi_trail = np.cos(theta0) * psi + np.sin(theta0) * D_prime
    F_theta0 = psi_trail @ H_matrix @ psi_trail.T
    partial_F = 2 * (D_prime @ H_matrix @ psi.T).real
    A1 = (F_0 - F_theta0 + 1/2 * partial_F * np.sin(2*theta0))/(1-np.cos(2*theta0))
    B1 = 1/2 * partial_F
    theta_new = 1/2 * np.arctan(B1 / A1)
    state_new = np.cos(theta_new) * psi + np.sin(theta_new) * D_prime
    return state_new

In [364]:
def pcg_ground_state_matrix(
    psi_set,
    psi0: np.ndarray,
    H_matrix: np.ndarray,
    precond: np.ndarray,
    max_iter: int = 50,
    tol: float = 1e-8,
    idx: int = 0 , ##说明是第几个波函数的迭代
 ) -> Tuple[np.ndarray, float, dict]:
    """
    对显式给出的矩阵 H_matrix 运行 PCG，输入与 `pcg_ground_state` 类似，输出归一化本征态、本征值以及收敛历史。
    """
    psi = psi0
    D_prev = None
    eta_prev = None
    theta0 = np.pi / 300
    for _ in range(max_iter):
        hpsi = H_matrix @ psi
        eig = np.real(np.vdot(psi, hpsi))
        print(np.linalg.norm(hpsi - eig * psi))
        G_prime = project_out_states(hpsi - eig * psi, psi_set, 'notself')
        eta_prime = project_out_states(G_prime * precond, psi_set, 'multi')
        if D_prev is None:
            D = eta_prime
        else:
            denom = np.vdot(G_prev, eta_prev).real
            denom = denom if abs(denom) > 1e-14 else 1e-14
            beta = np.vdot(eta_prime, G_prime).real / denom
            D = eta_prime + beta * D_prev
        O = project_out_states(D, psi, 'one')
        D_prime = normalize_state(O)
        psi = normalize_state(update_states(theta0, psi, D_prime, H_matrix))
        D_prev = D
        eta_prev = eta_prime
        G_prev = G_prime
        psi_set[: , idx] = psi
        history['res_norm'].append(np.linalg.norm(G_prime))
        if history['res_norm'][-1] < tol:
            print("Converged in {} iterations.".format(_+1))
            break
        if _ % 10 == 0:
            print(f"Iteration {_}: Residual norm = {history['res_norm'][-1]:.6e}")
    return psi

In [365]:
def run_scf(
    a0: float = 4.05,
    Ecut_eV: float = 120.0,
    Nk: int = 32,
    mix: float = 0.3,
    max_scf: int = 30,
    kvec=None,
    Z: float = 3.0,
    r_c: float = 0.6,
    r_gau: float = 1.0,
    projectors=None,
    nelec: float = 3.0,
    sigma_bohr: float = 1.0,
    pcg_max_iter: int = 60,
    pcg_tol: float = 1e-8,
    max_occ_per_state: float = 2.0,
) -> dict:

    """

    Γ 点 SCF：对每一次由 collect_ens 生成的 Hamiltonian 先做全对角化获取当前本征态集合，

    以最低本征矢作为 `pcg_ground_state_matrix` 的初始波函数，再用基态结果生成密度、线性混合并重新装配 Hamiltonian。

    循环 `max_scf` 轮后再对最终 Hamiltonian 执行一次对角化+PCG 精修，输出最终波函数及能量分解。

    """

    kvec_array = np.zeros(3) if kvec is None else np.asarray(kvec, dtype=float)

    basis = build_plane_wave_basis(a0=a0, Ecut_eV=Ecut_eV, Nk=Nk, kvec=kvec_array)

    precond = build_preconditioner(basis)

    def assemble_from_density(rho_grid: np.ndarray):

        """根据当前密度装配 collect_ens Hamiltonian，保证 Hermitian。"""

        ks_data = assemble_ks_hamiltonian(
            a0=a0,
            Ecut_eV=Ecut_eV,
            kvec=kvec_array,
            sigma_bohr=sigma_bohr,
            Nk_override=basis.Nk,
            Z_loc=Z,
            r_c=r_c,
            r_gau=r_gau,
            projectors=projectors,
            density_grid=rho_grid,
            basis_override=basis,
        )

        H = ks_data['H_total']

        H = 0.5 * (H + H.conj().T)

        return ks_data, H

    rho_init, _, _, _, _ = build_gaussian_density(a0=a0, Nk=basis.Nk, sigma_bohr=sigma_bohr)

    rho = rho_init * nelec
    print('初始总密度为：', np.sum(rho))
    ks_data_current, H_matrix_current = assemble_from_density(rho)

    results = {'eig_history': [], 'rho_history': [], 'diag_history': [], 'init': 'diag+pcg'}


    for scf_iter in range(1, max_scf + 1):

        eigvals, eigvecs = np.linalg.eigh(H_matrix_current)

        if eigvals.size == 0:

            raise RuntimeError('Hamiltonian 对角化失败：未获得任何本征值。')

        order = np.argsort(eigvals.real)

        eigvecs = eigvecs[:, order]

        eigvals = eigvals[order]

        print(f'SCF 第 {scf_iter} 步，基态本征值：', eigvals[0].real)

        rho_new = compute_density(eigvecs[:, 0], basis, nelec=2) + compute_density(eigvecs[:, 1], basis, nelec=1)
        rho_old = rho.copy()
        rho = mix * rho_new + (1 - mix) * rho
        print(np.linalg.norm(rho - rho_old))
        if np.linalg.norm(rho - rho_old) < 1e-6:
            print(f'SCF 在第 {scf_iter} 步收敛。')
            break
        ks_data_current , H_matrix_current = assemble_from_density(rho)
        # electrons_left = nelec

        # for idx in range(2): ## Al 的价电子有三个，对应只有两个轨道，因此只需迭代前两个本征态便能构造电荷密度
        #     if electrons_left <= 1e-10:
        #         break
        #     psi0 = eigvecs[:, idx]
        #     psi0 = normalize_state(psi0)
        #     psi = pcg_ground_state_matrix(
        #         eigvecs,
        #         psi0,
        #         H_matrix_current,
        #         precond,
        #         max_iter=pcg_max_iter,
        #         tol=pcg_tol,
        #         idx=idx,
        #         )

        #     occ = min(max_occ_per_state, electrons_left)
        #     rho_new += compute_density(psi, basis, nelec=occ)
        #     electrons_left -= occ
        #     eigvecs[:, idx] = psi
        # rho_new_normalized = rho_new / np.sum(rho_new) * nelec * basis.dV
        # rho = mix * rho_new_normalized + (1 - mix) * rho
        # if np.linalg.norm(rho - rho_new_normalized) < 1e-6:
        #     print(f'SCF 在第 {scf_iter} 步收敛。')
        #     break
        # ks_data_current , H_matrix_current = assemble_from_density(rho)

    eigvals_final, eigvecs_final = np.linalg.eigh(H_matrix_current)

    if eigvals_final.size == 0:

        raise RuntimeError('最终 Hamiltonian 对角化失败：未获得任何本征值。')
    
    order_final = np.argsort(eigvals_final.real)

    eigvals_final = eigvals_final[order_final]

    eigvecs_final = eigvecs_final[:, order_final]

    psi_final0 = eigvecs_final[:, 0]

    psi_final0 = normalize_state(psi_final0)

    psi_r = coeffs_to_real(psi_final0, basis)

    components = ks_data_current['components']

    electrons_left = nelec
    occupied_states = []
    for idx in range(eigvecs_final.shape[1]):
        if electrons_left <= 1e-10:
            break
        psi_i = normalize_state(eigvecs_final[:, idx])
        occ_i = min(max_occ_per_state, electrons_left)
        occupied_states.append((occ_i, psi_i))
        electrons_left -= occ_i
    if not occupied_states:
        raise RuntimeError('未能分配任何占据态，请检查 nelec 与能带数。')

    rho = None
    for occ_i, psi_i in occupied_states:
        contrib = compute_density(psi_i, basis, nelec=occ_i)
        rho = contrib if rho is None else rho + contrib

    E_kin = 0.0
    E_loc = 0.0
    E_nl = 0.0
    for occ_i, psi_i in occupied_states:
        E_kin += occ_i * np.real(np.vdot(psi_i, components['T'] @ psi_i))
        E_loc += occ_i * np.real(np.vdot(psi_i, components['V_ps_loc'] @ psi_i))
        E_nl += occ_i * np.real(np.vdot(psi_i, components['V_ps_nl'] @ psi_i))

    rho_G = fft_density(rho)

    Ggrid = build_G_grid(basis)

    E_H = compute_hartree_energy_from_nG(rho_G, Ggrid, basis.Omega)

    eps_x, _ = lda_exchange_correlation(rho)

    E_x = np.sum(eps_x * rho) * basis.dV

    E_tot = E_kin + E_loc + E_nl + E_H + E_x



    return {

        #'psi_coeffs': psi,
        'psi_coeffs' : psi_final0,

        'psi_real': psi_r,

        'rho': rho,

        'basis': basis,

        'ks_components': components,

        'energies': {

            'eigenvalue': eigvals_final[0],

            'E_kin': E_kin,

            'E_loc': E_loc,

            'E_nl': E_nl,

            'E_H': E_H,

            'E_x': E_x,

            'E_tot': E_tot,

        },

        'scf_history': results,

    }

In [366]:
def band_structure_along_path(
    scf_kwargs: dict,
    points_per_segment: int = 20,
    segments=None,
    pcg_kwargs=None,
    seed: int = 1,
    verbose: bool = False,
    freeze_density: bool = True,
    projectors=None,
    atom_positions=None,
    state_nelec: float = 1.0,
    num_bands: int = 4,
    ) -> dict:
    """
    先运行一次 Γ 点 SCF 获取参考密度，然后对每个 k 点调用 collect_ens 的 `assemble_ks_hamiltonian`
    直接装配 KS Hamiltonian，并用 PCG（显式矩阵版本）计算若干带能量；能量分解完全基于 collect_ens 返回的矩阵分量。
    """
    scf_kwargs = scf_kwargs.copy()
    sigma_bohr = scf_kwargs.get('sigma_bohr', 1.0)
    if verbose:
        print('Running reference SCF...')
    scf_result = run_scf(verbose=verbose, **scf_kwargs)
    reference_basis = scf_result['basis']
    rho_ref = scf_result['rho']
    if not freeze_density and verbose:
        print('freeze_density=False 当前仍使用参考密度生成势能（未做逐 k 自洽）。')
    density_for_ks = rho_ref

    segments = segments or [("Γ", "X"), ("X", "W"), ("W", "L")]
    k_samples, ticks = generate_kpath(reference_basis, segments=segments, points_per_segment=points_per_segment)
    params = {
        'a0': scf_kwargs.get('a0', 4.05),
        'Ecut_eV': scf_kwargs.get('Ecut_eV', 80.0),
        'Nk': scf_kwargs.get('Nk', 16),
        'Z': scf_kwargs.get('Z', 1.0),
        'r_c': scf_kwargs.get('r_c', 0.6),
        'r_gau': scf_kwargs.get('r_gau', 1.0),
    }
    if projectors is None:
        projectors = scf_kwargs.get('projectors')
    if projectors is None:
        projectors = [
            GaussianProjectorChannel(l=0, D_l=-0.5, r_cut=1.0),
            GaussianProjectorChannel(l=1, D_l=-0.1, r_cut=1.2),
        ]
    if atom_positions is None:
        atom_positions = [np.zeros(3)]  # collect_ens 目前内部仍使用原点单原子近似
    pcg_defaults = {'max_iter': 60, 'tol': 1e-9}
    if pcg_kwargs is None:
        pcg_kwargs = {}
    for key, val in pcg_defaults.items():
        pcg_kwargs.setdefault(key, val)
    rng = np.random.default_rng(seed)
    band_data = []
    num_bands = max(1, int(num_bands))

    for entry in k_samples:
        basis_k = build_plane_wave_basis(
            a0=params['a0'],
            Ecut_eV=params['Ecut_eV'],
            Nk=params['Nk'],
            kvec=entry['kvec'],
        )
        ks_data = assemble_ks_hamiltonian(
            a0=params['a0'],
            Ecut_eV=params['Ecut_eV'],
            kvec=entry['kvec'],
            sigma_bohr=sigma_bohr,
            Nk_override=basis_k.Nk,
            Z_loc=params['Z'],
            r_c=params['r_c'],
            r_gau=params['r_gau'],
            projectors=projectors,
            density_grid=density_for_ks,
            basis_override=basis_k,
        )
        H_matrix = 0.5 * (ks_data['H_total'] + ks_data['H_total'].conj().T)
        components = ks_data['components']
        precond = build_preconditioner(basis_k)
        locked_states = []
        label_display = entry['label'] or '-'
        for band_index in range(num_bands):
            attempts = 0
            while True:
                attempts += 1
                psi0 = rng.normal(size=len(basis_k.Gs)) + 1j * rng.normal(size=len(basis_k.Gs))
                try:
                    psi_k, eig, _ = pcg_ground_state_matrix(
                        psi0,
                        H_matrix,
                        precond,
                        max_iter=pcg_kwargs['max_iter'],
                        tol=pcg_kwargs['tol'],
                        locked_states=locked_states,
                    )
                    break
                except RuntimeError as exc:
                    if attempts < 8:
                        continue
                    raise RuntimeError(
                        f"无法为 k 点 {label_display} 的第 {band_index + 1} 条带收敛：{exc}"
                    ) from exc
            locked_states.append(psi_k)
            E_kin = np.real(np.vdot(psi_k, components['T'] @ psi_k))
            E_loc = np.real(np.vdot(psi_k, components['V_ps_loc'] @ psi_k))
            E_nl = np.real(np.vdot(psi_k, components['V_ps_nl'] @ psi_k))
            E_H = np.real(np.vdot(psi_k, components['V_H'] @ psi_k))
            E_x = np.real(np.vdot(psi_k, components['V_x'] @ psi_k))
            E_tot = E_kin + E_loc + E_nl + E_H + E_x
            band_data.append({
                'k_label': entry['label'],
                'path_pos': entry['path_pos'],
                'fractional_k': entry['frac'],
                'kvec_cart': entry['kvec'],
                'band_index': band_index,
                'eigenvalue': eig,
                'E_tot': E_tot,
                'state_nelec': state_nelec,
                'components': {
                    'E_kin': E_kin,
                    'E_loc': E_loc,
                    'E_nl': E_nl,
                    'E_H': E_H,
                    'E_x': E_x,
                },
            })
            if verbose:
                disp_label = entry['label'] or '-'
                print(f"k-point {disp_label:>2s} band {band_index + 1}: eig = {eig:.6f} Ha, E_tot = {E_tot:.6f} Ha")

    return {
        'reference': scf_result,
        'bands': band_data,
        'ticks': ticks,
        'segments': segments,
    }


In [367]:
if __name__ == '__main__':

    scf_params = dict(

        a0=4.05,

        Ecut_eV=400.0,

        Nk=32,

        mix=0.3,

        max_scf=80,

        nelec=3.0,

        kvec=[0.0, 0.0, 0.0],

        Z=3.0,

        r_c=0.7,

        r_gau=1.2,

        pcg_max_iter=80,

        pcg_tol=1e-6,

    )

    scf_result = run_scf(**scf_params)

    final_eig = scf_result['energies']['eigenvalue']

    print('\nΓ 点 SCF 收敛完成：')

    print(f"  最终本征值 (Ha): {final_eig:.6f}")

    print(f"  总能量 (Ha): {scf_result['energies']['E_tot']:.6f}")


初始总密度为： 2.9999999999999996
SCF 第 1 步，基态本征值： -42.846794430610586
0.033911065538166064
SCF 第 1 步，基态本征值： -42.846794430610586
0.033911065538166064
SCF 第 2 步，基态本征值： -40.009703449130186
0.023730336411729554
SCF 第 2 步，基态本征值： -40.009703449130186
0.023730336411729554
SCF 第 3 步，基态本征值： -37.97150391554008
0.01660408848149442
SCF 第 3 步，基态本征值： -37.97150391554008
0.01660408848149442
SCF 第 4 步，基态本征值： -36.542872571417206
0.011617011194559134
SCF 第 4 步，基态本征值： -36.542872571417206
0.011617011194559134
SCF 第 5 步，基态本征值： -35.54282080757656
0.008127341377737358
SCF 第 5 步，基态本征值： -35.54282080757656
0.008127341377737358
SCF 第 6 步，基态本征值： -34.843087900869506
0.005685680193537033
SCF 第 6 步，基态本征值： -34.843087900869506
0.005685680193537033
SCF 第 7 步，基态本征值： -34.35358545827473
0.003977408057265354
SCF 第 7 步，基态本征值： -34.35358545827473
0.003977408057265354
SCF 第 8 步，基态本征值： -34.01118712754328
0.0027823044244365256
SCF 第 8 步，基态本征值： -34.01118712754328
0.0027823044244365256
SCF 第 9 步，基态本征值： -33.77169998319008
0.001946247609957